# One-shot face recognition tutorial

This tutorial will use tensorflow v2 to create a face recognition model based on the one-shot architecture. <br/>

The steps are:

## Workspace setup

Let's import all the shared libraries that notebook uses section-wide

In [41]:
from os import path
from typing import Final

from tqdm.auto import tqdm
import matplotlib.pyplot as plt

Let's create a workspace with all the necessary folders in it

In [42]:
import os

WORKSPACE_DIR_PATH: Final[str] = 'tutorial_workspace'
DOWNLOADS_PATH: Final[str] = path.join(WORKSPACE_DIR_PATH, 'downloads')
ANCHORS_PATH: Final[str] = path.join(WORKSPACE_DIR_PATH, 'anchors')
POSITIVES_PATH: Final[str] = path.join(WORKSPACE_DIR_PATH, 'positives')
NEGATIVES_PATH: Final[str] = path.join(WORKSPACE_DIR_PATH, 'negatives')

os.makedirs(DOWNLOADS_PATH, exist_ok=True)
os.makedirs(ANCHORS_PATH, exist_ok=True)
os.makedirs(POSITIVES_PATH, exist_ok=True)
os.makedirs(NEGATIVES_PATH, exist_ok=True)

print('All necessary folders created')

All necessary folders created


## Data fetching

To train a model a dataset is necessary. <br/>
For that we will use the `LFW` dataset

### Fetch dataset

The dataset we will use named `LFW` and you can find it either in this [link](http://vis-www.cs.umass.edu/lfw/#download) or at the code below

In [4]:
import shutil
import requests

TAR_URL: Final[str] = 'http://vis-www.cs.umass.edu/lfw/lfw.tgz'
FILE_NAME: Final[str] = 'lfw.tgz'
file_path: str = path.join(DOWNLOADS_PATH, FILE_NAME)

if not path.exists(file_path):
    # Make an HTTP request within a context manager
    with requests.get(TAR_URL, stream=True) as r:
        
        # Check header to get content length, in bytes
        total_length = int(r.headers.get("Content-Length"))
        
        # Implement progress bar via tqdm
        with tqdm.wrapattr(r.raw, "read", total=total_length, desc="lfw.tar compressed dataset") as raw:
        
            # Save the output to a file
            with open(file_path, 'wb')as output:
                shutil.copyfileobj(raw, output)
else:
    print('Dataset already downloaded')

Dataset already downloaded


### Extract the dataset

As described above, the model needs a lot faces that not match to the anchors. <br/>
For that we will use the dataset we downloaded above.

So, we will extract the compressed files. and put them into the `NEGATIVES_PATH`.

In [30]:
import tarfile
from typing import IO, Iterable

file_path: str = path.join(DOWNLOADS_PATH, 'lfw.tgz')

with tarfile.open(file_path, 'r:gz') as compressed_file:
    # Filtering out all directories and non-jpg files
    all_members: Iterable[tarfile.TarInfo] = [m for m in compressed_file.getmembers() if '.jpg' not in m.name and m.name != None]
    
    for member in tqdm(iterable=all_members, total=len(all_members)):
        archive_filename: str = member.path.split('/')[-1]
        filename: str = path.join(NEGATIVES_PATH, archive_filename)
        
        buffer_reader: IO[bytes] = compressed_file.extractfile(member)
        with open(filename, 'wb') as file:
            file.write(buffer_reader.read())

100%|██████████| 18983/18983 [02:11<00:00, 144.00it/s]


### Collect positive and anchors data

Until now, we successfully loaded the `LFW` dataset into our workspace as negative shots (observations).

Now, let's dive into the positives.
So, we can get those observations in many ways. Like taking shots from the camera, use existing images. <br/>
Let's do both!

#### Taking shots from webcam

Before we take shots from the camera, we need to remember. The dataset has 250x250 pixels images. <br/>
For the simplicity of this project, we will preffer to use the exact same size with the new shots.

So remember, 250x250...

In [41]:
import cv2
import uuid

# In case it not working properly, you can try with different index
cap = cv2.VideoCapture(1)
while cap.isOpened():
    ret, frame = cap.read()

    # Cut a 250x250 pixels crop from the original feed
    start_x: int = 200
    start_y: int = 120
    height: int = start_y + 250
    width: int = start_x + 250
    frame = frame[start_x:width, start_y:height, :]

    # Collect anchor image
    if cv2.waitKey(1) & 0XFF == ord('a'):
        # Creating unique filename
        generated_name: str = f'{uuid.uuid1()}.jpg'
        img_name: str = path.join(ANCHORS_PATH, generated_name)
        cv2.imwrite(img_name, frame)

    # Collect positive image
    if cv2.waitKey(1) & 0XFF == ord('p'):
        # Creating unique filename
        generated_name: str = f'{uuid.uuid1()}.jpg'
        img_name: str = path.join(POSITIVES_PATH, generated_name)
        cv2.imwrite(img_name, frame)

    # Show image back to screen
    cv2.imshow('Image collection', frame)

    if cv2.waitKey(1) & 0XFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
print('Frame shape:', frame.shape)
plt.title('Last frame detected')
plt.imshow(frame)

#### Load data from existing images

Alternitivly, you can load the positive and anchors images from existing images files. <br/>
Just copy your images positive images into `$POSITIVES_PATH`, and the anchors will be copied into `ANCHORS_PATH`.

## Preprocess

All the data successfully collected. Let's start to prepare it to the training phase.

So, what we will do in this phase is:
* Create labeled dataset from the anchors, positives and negatives directories
* Split the dataset into training and validation sets

### Phase imports

In [43]:
from typing import Union
import tensorflow as tf
from tf_agents.typing.types import EagerTensor

### Data loading

Let's verify that all of the images we will going to use, having the same 'shape' (dimensions). <br/>
In case they are different, we will scale them to the apropriate dimensions.

In [44]:
DATASET_SIZE: Final[int] = 300

# Tensorflow will take all files matching to the pattern inside `list_files()`
anchor: tf.data.Dataset = tf.data.Dataset.list_files(f'{ANCHORS_PATH}/*.jpg').take(DATASET_SIZE)
positive: tf.data.Dataset = tf.data.Dataset.list_files(f'{POSITIVES_PATH}/*.jpg').take(DATASET_SIZE)
negative: tf.data.Dataset = tf.data.Dataset.list_files(f'{NEGATIVES_PATH}/*.jpg').take(DATASET_SIZE)

negative.as_numpy_iterator(), anchor.as_numpy_iterator(), positive.as_numpy_iterator()

(<tensorflow.python.data.ops.dataset_ops._NumpyIterator at 0x2be8d738f10>,
 <tensorflow.python.data.ops.dataset_ops._NumpyIterator at 0x2be8d7b5270>)

Example: This how you can iterate over a tensorlow Dataset class (in this case we will run over the anchors)

In [29]:
anchor.as_numpy_iterator().next()

b'tutorial_workspace\\anchors\\94e4272f-3853-11ed-b580-c85b76d2a6ae.jpg'

### Scale and resize the images

In [7]:
def preprocess(file_path: str):
    # Read image bytes from file path
    byte_image = tf.io.read_file(file_path)
    
    # Loading the bytes as image
    image = tf.io.decode_jpeg(byte_image)

    # Resize the image to 100x100 pixels
    image: EagerTensor = tf.image.resize(image, (100, 100))
    
    # Devide each pixel between (0 and 1) instead of (0 and 255)
    image /= 255.0

    return image

Just to be sure, let's verify that the pre process have done successfully.

In [ ]:
# Duplicate one of the lists 
_dup_anchor = anchor.as_numpy_iterator()
# Running the preprocess on the current image
img = preprocess(_dup_anchor.next())

print('Minimum value in tensor (should be 0):', img.numpy().min())
print('Max value in tensor (should be 1):', img.numpy().max())
print('Image shape (should be (100, 100, 3)):', img.numpy().shape)
plt.imshow(img)

### Create the labeled dataset

To train the model, we will need to give it:
* Positive observations (which means anchor image, positive image and result of 1).
* Negative observations (which means anchor image, negative and result of 0).

For that we will create two datasets, one for positive and one for negative. And concatenate them together.

In [31]:
positive_labels = tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor))) # Create a vector with shape of positive images
negative_labels = tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor))) # Create a vector with shape of negative images

# Telling the data loader to load each of those images with the proper label simultaneously 
positive_dataset = tf.data.Dataset.zip((anchor, positive, positive_labels))
negative_dataset = tf.data.Dataset.zip((anchor, negative, negative_labels))

# Concatenating the positive and negative datasets into a single dataset
dataset = positive_dataset.concatenate(negative_dataset)

#### Little example

In [40]:
sample = dataset.as_numpy_iterator()
dataset

<ConcatenateDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None))>

In [39]:
sample.next()

(b'tutorial_workspace\\anchors\\b2f63770-3853-11ed-b7c9-c85b76d2a6ae.jpg',
 b'tutorial_workspace\\positives\\6e10a761-3853-11ed-a140-c85b76d2a6ae.jpg',
 1.0)

### Build train and test dataset partitions

Let's load all the images from it's path, and put them the proper label

In [48]:
def preprocess_twin(input_img: str, validation_img: str, label: int):
    return (preprocess(input_img), preprocess(validation_img), label)

Let's test this function

In [55]:
example = sample.next()
res = preprocess_twin(*example)
res

(<tf.Tensor: shape=(100, 100, 3), dtype=float32, numpy=
 array([[[0.75686276, 0.7764706 , 0.7607843 ],
         [0.7607843 , 0.7794118 , 0.76666665],
         [0.7607843 , 0.7764706 , 0.77254903],
         ...,
         [0.66887254, 0.71593136, 0.7080882 ],
         [0.67156863, 0.7127451 , 0.7078431 ],
         [0.6754902 , 0.7147059 , 0.7107843 ]],
 
        [[0.75392157, 0.77254903, 0.75980395],
         [0.75465685, 0.7703431 , 0.76642156],
         [0.75784314, 0.7735294 , 0.7710784 ],
         ...,
         [0.6693627 , 0.7144608 , 0.70759803],
         [0.6723039 , 0.7129902 , 0.7083333 ],
         [0.67132354, 0.7115196 , 0.70465684]],
 
        [[0.75686276, 0.77254903, 0.7745098 ],
         [0.76004905, 0.7757353 , 0.7776961 ],
         [0.7607843 , 0.7764706 , 0.779902  ],
         ...,
         [0.6723039 , 0.7115196 , 0.70759803],
         [0.67156863, 0.7129902 , 0.702451  ],
         [0.67156863, 0.7147059 , 0.6990196 ]],
 
        ...,
 
        [[0.96911764, 0.9963235 

#### Example

So, after we have tested everything. Let's put all the things together. And build our dataloader pipline

In [56]:
dataset = dataset.map(preprocess_twin) # Running all over the dataset with the `preprocess_twin` function
dataset = dataset.cache() # Caching the images
dataset = dataset.shuffle(buffer_size=1024) # Shuffling the dataset
dataset

<ShuffleDataset element_spec=(TensorSpec(shape=(100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None))>

In [ ]:
samp = dataset.as_numpy_iterator().next()

print('Label is:', samp[2])
plt.imshow(samp[0])

In [ ]:
plt.imshow(samp[1])

#### Make training and test dataset partitions

Shared variables for this section

In [69]:
LEARNING_BATCH_SIZE: Final[int] = 16
LEARNING_PREFETCH_SIZE: Final[int] = 8

TRAIN_PARTITION_SIZE: Final[int] = round(len(dataset) * .7) # Get 70% of the dataset for training
TEST_PARTITION_SIZE: Final[int] = round(len(dataset) * .3) # Get 30% of the dataset for testing

The training partition setup:

In [70]:
training_data: tf.data.Dataset = dataset.take(TRAIN_PARTITION_SIZE)
training_data: tf.data.Dataset = training_data.batch(LEARNING_BATCH_SIZE) # Set batch size of 16
training_data: tf.data.Dataset = training_data.prefetch(LEARNING_PREFETCH_SIZE)
training_data

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(None, 100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

Setup testing partition

In [71]:
test_data = dataset.skip(TRAIN_PARTITION_SIZE) # Skipping of all the training partition data
test_data = test_data.take(TEST_PARTITION_SIZE)
test_data = test_data.batch(LEARNING_BATCH_SIZE)
test_data = test_data.prefetch(LEARNING_PREFETCH_SIZE)
test_data

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(None, 100, 100, None), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>